Colab for https://github.com/Mozer/YuE-extend
- It can extend any uploaded mp3
- uses exllama2 module
- flash_attention 2 is not supported for Tesla T4

# Install

In [ ]:
!pip install exllamav2==0.2.8
!git clone https://github.com/ocadaruma/YuE-extend
%cd YuE-extend
!pip install -r requirements.txt
!git clone https://huggingface.co/m-a-p/xcodec_mini_infer
!pip uninstall -y flash-attn

# Download models

In [ ]:
# download COT s1 + s2 models
# you can also download ICL model (optional)
!huggingface-cli download Alissonerdx/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw --local-dir workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw
!huggingface-cli download Alissonerdx/YuE-s2-1B-general-exl2-8.0bpw --local-dir workspace/models/YuE-s2-1B-general-exl2-8.0bpw

Don't start UI. Google colab doesn't like UIs. Use command line instead. You can edit lyrics and genre in Files on the left -> Yue-extend/prompt_egs/lyrics.txt (double click). Output mp3 will be saved in workspace/outputs/.

# lyrics2music

In [ ]:
# simple cot music generation. 1 verse (run_n_segments), 30s (max_new_tokens)
!python -u src/yue/infer.py --stage1_use_exl2 --stage1_model "workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw" --stage1_cache_size 30000 --stage1_cache_mode FP16 --stage2_use_exl2 --stage2_model "workspace/models/YuE-s2-1B-general-exl2-8.0bpw" --stage2_cache_size 50000 --stage2_cache_mode 'FP16' --genre_txt "prompt_egs/genre.txt" --lyrics_txt "prompt_egs/lyrics.txt" --run_n_segments 1 --output_dir workspace/outputs --cuda_idx 0 --seed 42 --max_new_tokens 3000 --basic_model_config xcodec_mini_infer/final_ckpt/config.yaml --resume_path xcodec_mini_infer/final_ckpt/ckpt_00360000.pth --config_path xcodec_mini_infer/decoders/config.yaml --vocal_decoder_path xcodec_mini_infer/decoders/decoder_131000.pth --inst_decoder_path xcodec_mini_infer/decoders/decoder_151000.pth --resume_after_n -1

# mp3 extend

Trim your mp3 up to 30s. Use "Files" menu on the left to upload your split pop.00001.Vocals.mp3 + pop.00001.Instrumental.mp3 to Yue-extend/prompt_egs. To split use: https://huggingface.co/spaces/theneos/audio-separator or other services.
Don't forget to update genre.txt and put full lyrics into lyrics.txt

In [ ]:
# mp3 extend cot. 1 verse (run_n_segments), 28s (max_new_tokens), 7s from mp3 (extend_mp3_end_time)
!python -u src/yue/infer.py --stage1_use_exl2 --stage1_model "workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw" --stage1_cache_size 25000 --stage1_cache_mode FP16 --stage2_use_exl2 --stage2_model "workspace/models/YuE-s2-1B-general-exl2-8.0bpw" --stage2_cache_size 50000 --stage2_cache_mode 'FP16' --genre_txt "prompt_egs/genre.txt" --lyrics_txt "prompt_egs/lyrics.txt" --run_n_segments 1 --output_dir workspace/outputs --cuda_idx 0 --seed 42 --max_new_tokens 2800 --basic_model_config xcodec_mini_infer/final_ckpt/config.yaml --resume_path xcodec_mini_infer/final_ckpt/ckpt_00360000.pth --config_path xcodec_mini_infer/decoders/config.yaml --vocal_decoder_path xcodec_mini_infer/decoders/decoder_131000.pth --inst_decoder_path xcodec_mini_infer/decoders/decoder_151000.pth --extend_mp3 --vocal_track_prompt_path "prompt_egs/pop.00001.Vocals.mp3" --instrumental_track_prompt_path "prompt_egs/pop.00001.Instrumental.mp3" --extend_mp3_end_time 7 --prompt_start_time 0 --prompt_end_time 8 --resume_after_n -1